[View in Colaboratory](https://colab.research.google.com/github/manan-arya/Analytics_Vidhya_Competition/blob/master/Analytic_Vidya.ipynb)

In [0]:
import numpy as np
import pandas as pd

In [4]:
!wget https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/train_NIR5Yl1.csv
!wget https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/test_8i3B3FC.csv

--2018-09-01 09:17:24--  https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/train_NIR5Yl1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12564851 (12M) [text/plain]
Saving to: ‘train_NIR5Yl1.csv.4’

train_NIR5Yl1.csv.4 100%[===================>]  11.98M  21.3MB/s    in 0.6s    

2018-09-01 09:17:25 (21.3 MB/s) - ‘train_NIR5Yl1.csv.4’ saved [12564851/12564851]

--2018-09-01 09:17:26--  https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/test_8i3B3FC.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting resp

In [0]:
data= pd.read_csv("train_NIR5Yl1.csv")
dataset = 'train_NIR5Yl1.csv'

test = pd.read_csv('test_8i3B3FC.csv')

In [6]:
data.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0


In [0]:
data.drop("ID",axis =1,inplace=True)

In [8]:
data.head()

,Tag,Reputation,Answers,Username,Views,Upvotes
0,a,3942.0,2.0,155623,7855.0,42.0
1,a,26046.0,12.0,21781,55801.0,1175.0
2,c,1358.0,4.0,56177,8067.0,60.0
3,a,264.0,3.0,168793,27064.0,9.0
4,c,4271.0,4.0,112223,13986.0,83.0


In [50]:
data_dummies=pd.get_dummies(data["Tag"])
data.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0


In [51]:
data = pd.concat([data,pd.get_dummies(data['Tag'], drop_first=True)], axis=1); data.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes,c,h,i,j,o,p,r,s,x
0,52664,a,3942.0,2.0,155623,7855.0,42.0,0,0,0,0,0,0,0,0,0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0,0,0,0,0,0,0,0,0,0
2,468453,c,1358.0,4.0,56177,8067.0,60.0,1,0,0,0,0,0,0,0,0
3,96996,a,264.0,3.0,168793,27064.0,9.0,0,0,0,0,0,0,0,0,0
4,131465,c,4271.0,4.0,112223,13986.0,83.0,1,0,0,0,0,0,0,0,0


In [46]:
data.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes,c,h,i,j,o,p,r,s,x
0,52664,a,3942.0,2.0,155623,7855.0,42.0,0,0,0,0,0,0,0,0,0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0,0,0,0,0,0,0,0,0,0
2,468453,c,1358.0,4.0,56177,8067.0,60.0,1,0,0,0,0,0,0,0,0
3,96996,a,264.0,3.0,168793,27064.0,9.0,0,0,0,0,0,0,0,0,0
4,131465,c,4271.0,4.0,112223,13986.0,83.0,1,0,0,0,0,0,0,0,0


In [0]:
data.drop("Tag",axis=1,inplace=True)

In [12]:
data.head()

,Reputation,Answers,Username,Views,Upvotes,c,h,i,j,o,p,r,s,x
0,3942.0,2.0,155623,7855.0,42.0,0,0,0,0,0,0,0,0,0
1,26046.0,12.0,21781,55801.0,1175.0,0,0,0,0,0,0,0,0,0
2,1358.0,4.0,56177,8067.0,60.0,1,0,0,0,0,0,0,0,0
3,264.0,3.0,168793,27064.0,9.0,0,0,0,0,0,0,0,0,0
4,4271.0,4.0,112223,13986.0,83.0,1,0,0,0,0,0,0,0,0


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X = data.drop('Upvotes', axis=1)
y = data['Upvotes']

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=69)

In [16]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(297040, 13)
(297040,)
(33005, 13)
(33005,)


In [0]:
import xgboost as xgb

xg_reg=xgb.XGBRegressor(objective='reg:linear',n_estimators=10,seed=150)
xg_reg.fit(X_train,y_train)
preds=xg_reg.predict(X_test)

In [0]:
from sklearn.metrics import mean_squared_error
rmse=np.sqrt(mean_squared_error(y_test,preds))

In [19]:
print("RMSE : %f"% (rmse))

RMSE : 2276.641399


In [20]:
#tuning and boosting of xg boost

DM_train = xgb.DMatrix(data=X_train,label=y_train)
DM_test = xgb.DMatrix(data=X_test,label=y_test)
params = {"booster":"gblinear","objective":"reg:linear"}
xg_reg=xgb.train(params=params,dtrain=DM_train,num_boost_round=10)

preds_2=xg_reg.predict(DM_test)

rmse_2=np.sqrt(mean_squared_error(y_test,preds_2))

print("new RMSE= %f" %rmse_2)


new RMSE= 3296.124775


In [21]:
rmse

2276.6413991351656

In [22]:
rmse_2

3296.12477505686

### Trying to apply grid search cv

In [108]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline


steps = [('clf', xgb.XGBRegressor())]


clf_param_grid = {
    'clf__max_depth' : [5,10,20],
    'clf__learning_rate' : [0.01],
    'clf__n_jobs' : [4],
    #'clf__n_estimators' : [50, 100,]
    
}


xgbclf = Pipeline(steps)

X_new = X.copy()
# X_new.drop('ID',axis=1,inplace=True)
estimator = GridSearchCV(xgbclf, param_grid=clf_param_grid ,cv =4, verbose=2, scoring="neg_mean_squared_error")
estimator.fit(X_new,y)

print(estimator.best_params_)
print(np.sqrt(-1*estimator.best_score_))

Fitting 4 folds for each of 3 candidates, totalling 12 fits
[CV] clf__learning_rate=0.01, clf__max_depth=5, clf__n_jobs=4 ........
[CV]  clf__learning_rate=0.01, clf__max_depth=5, clf__n_jobs=4, total=  12.6s
[CV] clf__learning_rate=0.01, clf__max_depth=5, clf__n_jobs=4 ........


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.1s remaining:    0.0s


[CV]  clf__learning_rate=0.01, clf__max_depth=5, clf__n_jobs=4, total=  13.6s
[CV] clf__learning_rate=0.01, clf__max_depth=5, clf__n_jobs=4 ........
[CV]  clf__learning_rate=0.01, clf__max_depth=5, clf__n_jobs=4, total=  13.6s
[CV] clf__learning_rate=0.01, clf__max_depth=5, clf__n_jobs=4 ........
[CV]  clf__learning_rate=0.01, clf__max_depth=5, clf__n_jobs=4, total=  13.7s
[CV] clf__learning_rate=0.01, clf__max_depth=10, clf__n_jobs=4 .......
[CV]  clf__learning_rate=0.01, clf__max_depth=10, clf__n_jobs=4, total=  28.4s
[CV] clf__learning_rate=0.01, clf__max_depth=10, clf__n_jobs=4 .......
[CV]  clf__learning_rate=0.01, clf__max_depth=10, clf__n_jobs=4, total=  29.3s
[CV] clf__learning_rate=0.01, clf__max_depth=10, clf__n_jobs=4 .......
[CV]  clf__learning_rate=0.01, clf__max_depth=10, clf__n_jobs=4, total=  29.0s
[CV] clf__learning_rate=0.01, clf__max_depth=10, clf__n_jobs=4 .......
[CV]  clf__learning_rate=0.01, clf__max_depth=10, clf__n_jobs=4, total=  29.0s
[CV] clf__learning_rate=

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  8.6min finished


{'clf__learning_rate': 0.01, 'clf__max_depth': 5, 'clf__n_jobs': 4}
2067.826276045736


# Attention !!!
#### i have just tried a bit more tuning....
initially i took num_boost_round=200 so it didn't run on my pc,so i reduced it to 2 then only rmse was 2368.but when i just increased to 20 th result is here!
So, try to run it at 200 rounds on gpu or even more. Also, try to change the nfolds.

In [105]:
review_data_matrix=xgb.DMatrix(data=X,label=y)

untuned_params = {"objective":"reg:linear"}

tuned_cv_rmse = xgb.cv(dtrain = review_data_matrix,params=untuned_params, nfold=7, num_boost_round=200, metrics="rmse",as_pandas=True, seed=123)

print(type(tuned_cv_rmse))

print("Tuned rmse: %f" %((tuned_cv_rmse["test-rmse-mean"]).tail(1)))


<class 'pandas.core.frame.DataFrame'>
Tuned rmse: 1387.316842


In [107]:
xgb.train(X_train,y_train)

TypeError: ignored

In [106]:
tuned_cv_rmse

,test-rmse-mean,test-rmse-std,train-rmse-mean,train-rmse-std
0,2810.816249,640.576767,2748.033901,93.865772
1,2307.564767,570.361063,2136.249337,65.832224
2,1944.782837,484.659488,1691.847412,50.062055
3,1741.683001,467.528235,1371.963152,38.532558
4,1598.643468,449.350080,1143.333252,32.975317
5,1534.369184,438.720536,977.872254,31.391180
6,1475.872498,426.911525,860.712882,28.310260
7,1437.587943,410.863533,775.987374,26.606217
8,1423.162920,398.336829,717.976318,25.722909
9,1402.554007,376.568150,674.757812,26.280501


## trying more tuning methods

I didn't found this method benificial, so just go through it and if possible try hyperparameter tuning.that's it

In [100]:
tuned_params={"objective":"reg:linear",'colsample_bytree':0.3,'learning_rate':0.1,'max_depth':5}

new_tuned_cv_rmse = xgb.cv(dtrain=review_data_matrix, params=tuned_params, nfold=5, num_boost_round=20, metrics="rmse",as_pandas=True, seed=123)
print("Tuned rmse: %f" %((new_tuned_cv_rmse["test-rmse-mean"]).tail(1)))


Tuned rmse: 2976.052100


# SVR

In [0]:
from sklearn.svm import SVR

svr_rbf = SVR(kernel = 'rbf', C = 1e2, gamma = 0.1)

svr_rbf.fit(X_train,y_train)
svr_rbf.predict(X_test)
svr_rbf.score(X_test,y_test)

In [0]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

steps = [('scale',StandardScaler()),('dt',DecisionTreeRegressor())]

X_train_new = X_train.drop('Username',axis = 1)
X_test_new = X_test.drop('Username',axis = 1)

dtr = Pipeline(steps)

dtr.fit(X_train_new,y_train)
preds_dt = dtr.predict(X_test_new)

rmse_dt = np.sqrt(mean_squared_error(preds_dt,y_test))

In [34]:
rmse_dt

2054.95235126629

In [25]:
X_train

,Reputation,Answers,Username,Views,c,h,i,j,o,p,r,s,x
154242,187.0,1.0,96073,120.0,0,0,0,1,0,0,0,0,0
164698,982.0,2.0,80202,5204.0,0,0,0,0,0,0,0,0,0
268304,550.0,3.0,153756,16830.0,1,0,0,0,0,0,0,0,0
293794,1627.0,2.0,54644,2097.0,1,0,0,0,0,0,0,0,0
214564,344.0,7.0,58117,25084.0,1,0,0,0,0,0,0,0,0
20537,25377.0,9.0,57895,124797.0,0,0,0,1,0,0,0,0,0
76345,896.0,4.0,23223,19469.0,0,0,0,0,0,0,0,0,0
139715,18674.0,8.0,125424,71268.0,0,0,0,0,0,0,0,1,0
175706,14612.0,2.0,83294,934.0,1,0,0,0,0,0,0,0,0
184263,581.0,2.0,49367,45752.0,0,0,0,0,0,0,0,0,0


In [0]:
from sklearn.linear_model import LinearRegression
steps = [('scale',StandardScaler()),('lre',LinearRegression())]

X_train_new = X_train.drop('Username',axis = 1)
X_test_new = X_test.drop('Username',axis = 1)

lr = Pipeline(steps)

lr.fit(X_train_new,y_train)
preds_lr = lr.predict(X_test_new)

rmse_lr = np.sqrt(mean_squared_error(preds_lr,y_test))

In [36]:
rmse_lr

3294.848060148972

# Random Forest

In [28]:
data.head()

,Reputation,Answers,Username,Views,Upvotes,c,h,i,j,o,p,r,s,x
0,3942.0,2.0,155623,7855.0,42.0,0,0,0,0,0,0,0,0,0
1,26046.0,12.0,21781,55801.0,1175.0,0,0,0,0,0,0,0,0,0
2,1358.0,4.0,56177,8067.0,60.0,1,0,0,0,0,0,0,0,0
3,264.0,3.0,168793,27064.0,9.0,0,0,0,0,0,0,0,0,0
4,4271.0,4.0,112223,13986.0,83.0,1,0,0,0,0,0,0,0,0


In [0]:
def subsample(dataset, ratio):
	sample = list()
	n_sample = round(len(dataset) * ratio)
	while len(sample) < n_sample:
		index = randrange(len(dataset))
		sample.append(dataset[index])
	return sample


In [0]:
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
	trees = list()
	for i in range(n_trees):
		sample = subsample(train, sample_size)
		tree = build_tree(sample, max_depth, min_size, n_features)
		trees.append(tree)
	predictions = [bagging_predict(trees, row) for row in test]
	return(predictions)

In [31]:
y.head()

0      42.0
1    1175.0
2      60.0
3       9.0
4      83.0
Name: Upvotes, dtype: float64

In [32]:

n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features=
for n_trees in [1, 5, 10]:
	scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
	print('Trees: %d' % n_trees)
	print('Scores: %s' % scores)
	print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))


SyntaxError: ignored

# Submission Time!! :-)

In [53]:
test.head()

,ID,Tag,Reputation,Answers,Username,Views
0,366953,a,5645.0,3.0,50652,33200.0
1,71864,c,24511.0,6.0,37685,2730.0
2,141692,i,927.0,1.0,135293,21167.0
3,316833,i,21.0,6.0,166998,18528.0
4,440445,i,4475.0,10.0,53504,57240.0


In [54]:
test = pd.concat([test,pd.get_dummies(test['Tag'], drop_first=True)], axis=1)
test.head()

,ID,Tag,Reputation,Answers,Username,Views,c,h,i,j,o,p,r,s,x
0,366953,a,5645.0,3.0,50652,33200.0,0,0,0,0,0,0,0,0,0
1,71864,c,24511.0,6.0,37685,2730.0,1,0,0,0,0,0,0,0,0
2,141692,i,927.0,1.0,135293,21167.0,0,0,1,0,0,0,0,0,0
3,316833,i,21.0,6.0,166998,18528.0,0,0,1,0,0,0,0,0,0
4,440445,i,4475.0,10.0,53504,57240.0,0,0,1,0,0,0,0,0,0


In [55]:
type(preds)

numpy.ndarray

In [64]:
test_new.head()

,ID,Reputation,Answers,Views,c,h,i,j,o,p,r,s,x
0,366953,5645.0,3.0,33200.0,0,0,0,0,0,0,0,0,0
1,71864,24511.0,6.0,2730.0,1,0,0,0,0,0,0,0,0
2,141692,927.0,1.0,21167.0,0,0,1,0,0,0,0,0,0
3,316833,21.0,6.0,18528.0,0,0,1,0,0,0,0,0,0
4,440445,4475.0,10.0,57240.0,0,0,1,0,0,0,0,0,0


In [0]:
test.drop('Tag', axis =1, inplace = True)

In [0]:
ID=test_new['ID']

test.drop('ID',axis=1,inplace = True)

In [69]:
X.head()

,Reputation,Answers,Views,c,h,i,j,o,p,r,s,x
0,3942.0,2.0,7855.0,0,0,0,0,0,0,0,0,0
1,26046.0,12.0,55801.0,0,0,0,0,0,0,0,0,0
2,1358.0,4.0,8067.0,1,0,0,0,0,0,0,0,0
3,264.0,3.0,27064.0,0,0,0,0,0,0,0,0,0
4,4271.0,4.0,13986.0,1,0,0,0,0,0,0,0,0


In [71]:
X_new = X.drop('Username',axis = 1,inplace = True)
test_new = test.drop('Username',axis = 1, inplace = True)

ValueError: ignored

In [73]:
test.head()

,Reputation,Answers,Views,c,h,i,j,o,p,r,s,x
0,5645.0,3.0,33200.0,0,0,0,0,0,0,0,0,0
1,24511.0,6.0,2730.0,1,0,0,0,0,0,0,0,0
2,927.0,1.0,21167.0,0,0,1,0,0,0,0,0,0
3,21.0,6.0,18528.0,0,0,1,0,0,0,0,0,0
4,4475.0,10.0,57240.0,0,0,1,0,0,0,0,0,0


In [0]:
steps = [('scale',StandardScaler()),('dt',DecisionTreeRegressor())]


dtr = Pipeline(steps)

dtr.fit(X,y)
preds_final = dtr.predict(test)

Sub_data=preds_final

submission = pd.DataFrame({'ID':ID,'Upvotes':preds_final})

In [101]:
submission

,ID,Upvotes
0,366953,217.0
1,71864,67.0
2,141692,11.0
3,316833,25.0
4,440445,430.0
5,3514,7.0
6,331420,12.0
7,406540,88.0
8,227379,41.0
9,331687,61.0


In [84]:
preds_final

array([260.,  67.,  11., ...,  36.,   6.,  10.])

In [102]:
ID

0         366953
1          71864
2         141692
3         316833
4         440445
5           3514
6         331420
7         406540
8         227379
9         331687
10         32784
11        470377
12        338103
13         29883
14        218228
15         70945
16         14328
17        259947
18        346378
19        384621
20        425418
21        131224
22        365408
23        218626
24        331722
25        310969
26        307391
27         14072
28        238694
29        213841
           ...  
141418    158414
141419     29710
141420    370802
141421    264407
141422    159508
141423     18508
141424      9046
141425    358118
141426    470852
141427    106824
141428    371009
141429    434645
141430    461727
141431    208029
141432    133474
141433    404689
141434    113320
141435    403339
141436     65118
141437    142461
141438    338255
141439     15418
141440    267690
141441    108226
141442    166359
141443     47187
141444    329126
141445    2823

In [0]:
submission.to_csv('hackathon.csv', index=False)
from google.colab import files 
files.download('hackathon.csv')